In [2]:
#Kevin Song Ngy Tea
#Thanh-Liem Huynh

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
!pip install category_encoders
!pip install scikit-optimize


  Using cached category_encoders-2.6.3-py2.py3-none-any.whl (81 kB)


In [145]:
df_train = pd.read_csv("ift6390_traindata.csv")
df_meta = pd.read_csv("ift6390_metadata.csv")
df_merge = pd.merge(df_train, df_meta, on="building_id")
df_merge.sort_values(by="row_id", inplace=True)



In [ ]:
df_merge.dtypes

In [146]:
# There are some empty entries? Let's quickly look at how many there are
# Il y a des entrées vides ? Jetons rapidement un coup d'œil pour voir combien il en manque

def compute_nan(data):
    num_rows = data.shape[0]
    for var in data.columns:
        num_nans = len(data[data[var].isnull()])
        percentage = np.round(num_nans / num_rows * 100, 2)
        print('{:<20} {:>10}% nan data'.format(var, percentage))

In [147]:
import random

def random_bin(proportion, Binary_1, Binary_0):

  random_number = random.randint(1, 10)

  if(random_number <= proportion):
    return Binary_1
  else:
    return Binary_0


In [148]:
average_age = round(df_merge["age"].mean())
print("average age: " + str(average_age))

average_clo = df_merge["clo"].mean()
print("average clo: " + str(average_clo))

average_met = df_merge["met"].mean()
print("average met: " + str(average_met))

average_height = df_merge["ht"].mean()
print("average height: " + str(average_height))

average_weight = df_merge["wt"].mean()
print("average weight: " + str(average_weight))

average_t_out = df_merge["t_out"].mean()

average_rh_out = df_merge["rh_out"].mean()

average_rh = df_merge["rh"].mean()
print("average rh: " + str(average_rh))

average_curtain = int(df_merge["blind_curtain"].mean())
print("average curtain: " + str(average_curtain))

average_door = int(df_merge["door"].mean())
print("average door: " + str(average_door))

average_fan = int(df_merge["fan"].mean())
print("average fan: " + str(average_fan))

average_vel = df_merge["vel"].mean()


print(df_merge.shape)
t_out_grouped = df_merge.groupby(["climate", "season"])["t_out"].mean()
print(t_out_grouped.shape)


average age: 33
average clo: 0.6811800418782404
average met: 1.2103452262110785
average height: 1.6689524101934297
average weight: 65.37748449955714
average rh: 47.227948569318976
average curtain: 0
average door: 0
average fan: 0
(60110, 23)
(62,)


In [149]:
def OHE(data):
  ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False).set_output(transform="pandas")

  #gender
  if 'gender' in data.columns:
    ohetransform1 = ohe.fit_transform(data[["gender"]])
    data = pd.concat([data, ohetransform1], axis=1).drop(columns=["gender"])

  #season
  if 'season' in data.columns:
    ohetransform2 = ohe.fit_transform(data[["season"]])
    data = pd.concat([data, ohetransform2], axis=1).drop(columns=["season"])

  #climate
  if 'climate' in data.columns:
    ohetransform3 = ohe.fit_transform(data[["climate"]])
    data = pd.concat([data, ohetransform3], axis=1).drop(columns=["climate"])

  #cooling type
  if 'cooling_type' in data.columns:
    ohetransform4 = ohe.fit_transform(data[["cooling_type"]])
    data = pd.concat([data, ohetransform4], axis=1).drop(columns=["cooling_type"])

  return data

In [150]:
def data_imputation2(data):
  data["age"].fillna(average_age, inplace=True)

  if 'gender' in data.columns:
    data["gender"].fillna(random_bin(5, "male", "female"), inplace=True)

  data["clo"].fillna(average_clo, inplace=True)
  data["met"].fillna(average_met, inplace=True)

  #rh
  if 'climate' in data.columns:
    data['rh'] = data.groupby(['climate', 'season'])['rh'].transform(lambda x: x.fillna(x.mean()))
  else:
    data["rh"].fillna(average_rh, inplace=True)

  if 'cooling_type' in data.columns:
    data["cooling_type"].fillna("mixed mode", inplace=True)

  columns_to_drop = ['blind_curtain', 'door', 'fan', 'tr', 'tg', 'window', 'ht', 'wt']
  if all(col in data.columns for col in columns_to_drop):
    data.drop(columns=columns_to_drop, inplace=True)

  #t_out
  if 'climate' in data.columns:
    data['t_out'] = data.groupby(['climate', 'season'])['t_out'].transform(lambda x: x.fillna(x.mean()))
    data['t_out'] = data.groupby(['climate'])['t_out'].transform(lambda x: x.fillna(x.mean()))
    data['t_out'] = data.groupby(['season'])['t_out'].transform(lambda x: x.fillna(x.mean()))
  else:
    data["t_out"].fillna(average_t_out, inplace=True)

  #rh_out
  if 'climate' in data.columns:
    data['rh_out'] = data.groupby(['climate', 'season'])['rh_out'].transform(lambda x: x.fillna(x.mean()))
    data['rh_out'] = data.groupby(['climate'])['rh_out'].transform(lambda x: x.fillna(x.mean()))
    data['rh_out'] = data.groupby(['season'])['rh_out'].transform(lambda x: x.fillna(x.mean()))
  else:
    data["rh_out"].fillna(average_rh_out, inplace=True)

  #vel
  if 'cooling_type' in data.columns:
    data['vel'] = data.groupby(['cooling_type'])['vel'].transform(lambda x: x.fillna(x.mean()))
  else:
    data["vel"].fillna(average_vel, inplace=True)


  data = OHE(data)

  return data




In [151]:
df_merge = data_imputation2(df_merge)
compute_nan(df_merge)


row_id                      0.0% nan data
age                         0.0% nan data
clo                         0.0% nan data
met                         0.0% nan data
t_out                       0.0% nan data
rh_out                      0.0% nan data
ta                          0.0% nan data
rh                          0.0% nan data
vel                         0.0% nan data
building_id                 0.0% nan data
thermal_sensation           0.0% nan data
gender_female               0.0% nan data
gender_male                 0.0% nan data
season_autumn               0.0% nan data
season_cool/dry             0.0% nan data
season_hot/wet              0.0% nan data
season_spring               0.0% nan data
season_summer               0.0% nan data
season_winter               0.0% nan data
climate_continental subarctic        0.0% nan data
climate_cool-summer mediterranean        0.0% nan data
climate_desert (hot arid)        0.0% nan data
climate_hot arid            0.0% nan data
climate

In [152]:
test_data = pd.read_csv('./test.csv')
test_data = pd.merge(test_data, df_meta, on="building_id")
test_data.sort_values(by="row_id", inplace=True)

test_data = data_imputation2(test_data)
test_data.drop(columns=["gender_undefined"], inplace=True)
X_test = test_data

compute_nan(X_test)

row_id                      0.0% nan data
age                         0.0% nan data
clo                         0.0% nan data
met                         0.0% nan data
t_out                       0.0% nan data
rh_out                      0.0% nan data
ta                          0.0% nan data
rh                          0.0% nan data
vel                         0.0% nan data
building_id                 0.0% nan data
gender_female               0.0% nan data
gender_male                 0.0% nan data
season_autumn               0.0% nan data
season_cool/dry             0.0% nan data
season_hot/wet              0.0% nan data
season_spring               0.0% nan data
season_summer               0.0% nan data
season_winter               0.0% nan data
climate_continental subarctic        0.0% nan data
climate_cool-summer mediterranean        0.0% nan data
climate_desert (hot arid)        0.0% nan data
climate_hot arid            0.0% nan data
climate_hot desert          0.0% nan data
climate

In [153]:
from sklearn.model_selection import train_test_split

thermal_sensation_column = df_merge.pop("thermal_sensation")  # Extract the column

# Reinsert the column at the desired position (at the end)
df_merge["thermal_sensation"] = thermal_sensation_column

X_train = df_merge.iloc[:,:-1]
y_train = df_merge.iloc[:,-1]

y_train += 3

In [154]:
#Pipeline
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier

estimators = [
    ("encorder", TargetEncoder()),
    ("clf", XGBClassifier(ranbdom_state=8))
]
pipe = Pipeline(steps=estimators)
pipe

Pipeline(steps=[('encorder', TargetEncoder()),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ranbdom_state=8, ...))])

In [155]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

search_space = {
    "clf__max_depth": Integer(2,8),
    "clf__learning_rate": Real(0.001, 0.5, prior="log-uniform"),
    "clf__subsample": Real(0.5, 1.0),
    "clf__colsample_bytree": Real(0.5, 1.0),
    "clf__colsample_bylevel": Real(0.5, 1.0),
    "clf__colsample_bynode": Real(0.5, 1.0),
    "clf__reg_alpha": Real(0.0, 10.0),
    "clf__reg_lambda": Real(0.0, 10.0),
    "clf__gamma": Real(0.0, 10.0),
}

opt = BayesSearchCV(pipe, search_space, cv=3, n_iter=10, scoring="accuracy", random_state=8)

In [ ]:
opt.fit(X_train, y_train)

In [75]:
opt.best_estimator_

Pipeline(steps=[('encorder', TargetEncoder(cols=[])),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=0.9425384185492701,
                               colsample_bynode=0.9095956806239844,
                               colsample_bytree=0.706128679361455, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=1.6598135411398998,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.40582798722369146, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=7,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=None,
                               n_jobs=None, num_parallel_tree=None,
                               ranbdom_state=8, ...))])

In [158]:
opt.best_score_

0.4448677645829495

In [159]:
opt.score(X_train, y_train)

0.45080685410081517

In [160]:
# Prédire les classes sur l'ensemble de test

y_pred_test = opt.predict(X_test) - 3
y_pred_test_corrected = y_pred_test
print(y_pred_test_corrected)

[0 0 2 ... 0 0 0]


In [ ]:
#Nos résults produits par XGBoost
new_predictions = np.zeros((test_data.shape[0], 2))
np_test = test_data.to_numpy()
for i, d in enumerate(np_test):
  new_predictions[i, 0] = int(d[0])
  new_predictions[i, 1] = y_pred_test_corrected[i]

new_sample_submission = pd.DataFrame(data=new_predictions,
                                     columns=['row_id', 'thermal_sensation'])
new_sample_submission

In [162]:
# save into csv
#df_merge.to_csv("./df_merge.csv", index=False)
new_sample_submission.to_csv('./new_sample_submission28.csv', index=False)